## Computational Macroeconomics Assignment 5
Paula Beck

handed in: 29.01.2022

In [211]:
# some preparation

# import packages
from pathlib import Path
import numpy as np
import pandas as pd
from statsmodels.formula.api import ols


# set path
wd = Path()

### Step 1: download data and save
downloaded on 17.01.2022

### Step 2: load data

In [212]:
# load consumer expectation data

# instead of changing the excel file manually, I load the specific data directly in python by loading the explicite sheet and skipping the first three rows. Also, I only load the first three columns as those are the one we need.
exp = pd.read_excel(wd/'data'/'FRBNY-SCE-Data.xlsx', sheet_name = 'Inflation expectations', skiprows=3, usecols=[0,1,2])

exp.rename(columns={'Unnamed: 0':'date'}, inplace=True)

print(exp.head())

     date  Median one-year ahead expected inflation rate  \
0  201306                                       3.090884   
1  201307                                       3.162522   
2  201308                                       3.395071   
3  201309                                       3.367290   
4  201310                                       3.174733   

   Median three-year ahead expected inflation rate  
0                                         3.416846  
1                                         3.310545  
2                                         3.799491  
3                                         3.546918  
4                                         3.196597  


In [213]:
# load inflation data

inf = pd.read_csv(wd/'data'/'CPALTT01USM659N.csv', sep = ',', decimal ='.', names=['date', 'inflation'], header=0)

print(inf.head())

         date  inflation
0  1960-01-01   1.034483
1  1960-02-01   1.730104
2  1960-03-01   1.730104
3  1960-04-01   1.724138
4  1960-05-01   1.724138


### Step 3: convert to datetime

In [214]:
# first look what data types they have
print('---data types of inflation data------------')
print(inf.dtypes)
print('---data types of expectation data----------')
print(exp.dtypes)

---data types of inflation data------------
date          object
inflation    float64
dtype: object
---data types of expectation data----------
date                                                 int64
Median one-year ahead expected inflation rate      float64
Median three-year ahead expected inflation rate    float64
dtype: object


In [215]:
# convert date column of inflation dataframe to datetime
inf.date  = pd.to_datetime(inf.date)
# set frequency to montly
inf.date = pd.PeriodIndex(inf.date, freq='m')

# convert the date column of the expectation dataframe to a string and add '01', then convert to datetime
exp.date = exp.date.astype('str') + '01'
exp.date = pd.to_datetime(exp.date, format = '%Y%m%d')
# set frequency to montly
exp.date = pd.PeriodIndex(exp.date, freq='m')


In [216]:
# set date as index
inf = inf.set_index('date')
exp = exp.set_index('date')

### Step 4: rename columns

In [217]:
exp.rename(columns={exp.columns[0]:'med1y',exp.columns[1]:'med3y'}, inplace=True)

### Step 5: merge dataframes

In [218]:
# merge expectation data to inflation dataframe based on the index
# inner merge such that only those rows remain that are not NA for both data sources
df = inf.merge(exp, on='date', how='inner')

### Step 6: calculate forecast errors and forecast revision

In [219]:
####### check again !!!!!

df['med1y_shift'] = df['med1y'].shift(1)
df['med3y_shift'] = df['med3y'].shift(3)

df['ferror'] = df.inflation - df.med1y_shift
df['frevision'] = df.med1y_shift - df.med3y_shift

### Step 7: drop all columns except from forecast errors and forecast revision

In [220]:
df = df[['ferror', 'frevision']]

### Step 8: remove NAN values

In [221]:
df = df.dropna()
df.head()

,ferror,frevision
date,,
2013-09,-2.210146,-0.021775
2013-10,-2.403677,0.056744
2013-11,-1.937660,-0.624758
2013-12,-1.694861,-0.350321
2014-01,-1.558893,-0.058756


### Step 9: estimate equation (1)

- equation: ferror = constant + frevision + error
- set cov_type = 'HAC'
- set cov_kwds = {'maxlags':3}

In [226]:
model = ols(formula='ferror ~ frevision', data=df)
fit = model.fit(cov_type = 'HAC', cov_kwds = {'maxlags':3})
fit.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 ferror   R-squared:                       0.183
Model:                            OLS   Adj. R-squared:                  0.174
Method:                 Least Squares   F-statistic:                     23.15
Date:                Mon, 17 Jan 2022   Prob (F-statistic):           5.56e-06
Time:                        15:50:23   Log-Likelihood:                -126.59
No. Observations:                  98   AIC:                             257.2
Df Residuals:                      96   BIC:                             262.4
Df Model:                           1                                         
Covariance Type:                  HAC                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -1.1840      0.160     -7.423      0.000      -1.497      -0.871
frevision      1.0989      0.228      4.812      0.000       0.651       1.547
==============================================================================
Omnibus:                        1.044   Durbin-Watson:                   0.291
Prob(Omnibus):                  0.593   Jarque-Bera (JB):                1.136
Skew:                          -0.223   Prob(JB):                        0.567
Kurtosis:                       2.717   Cond. No.                         2.64
==============================================================================

Notes:
[1] Standard Errors are heteroscedasticity and autocorrelation robust (HAC) using 3 lags and without small sample correction
"""

### Step 10: Interpretation